
<h2 style="text-align:center;font-size:200%;;">Лабораторная работа №2 </h2>
<h3  style="text-align:center;"><span class="label label-success">Машинное обучение на больших данных</span>


## Цель и задачи работы:
1. Познакомиться с базовыми алгоритмами машинного обучения;
2. Познакомиться с реализацией машинного обучения в библиотеке Spark ML.
3. Получить навыки разработки программного обеспечения для анализа данных с
использованием pyspark.

## Выполнение работы
### 1. Импорт необходимых библиотек и датасета

In [1]:
import numpy as np 
import pandas as pd
import os
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import seaborn as sns

from pyspark.ml.regression import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, VectorIndexer, MinMaxScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator

spark = SparkSession.builder.master("local[*]").getOrCreate()

print(os.listdir("."))

['.git', '.ipynb_checkpoints', 'advanced-pyspark-for-exploratory-data-analysis.ipynb', 'data', 'dataindiv', 'df2.csv', 'docker-compose.yml', 'lab1.ipynb', 'machine-learning-with-spark (2).ipynb', 'model', 'README.md', 'SOBD2023-1-lab.ipynb', 'SOBD2023-2-lab.ipynb']


### Загрузка данных
Мною был использован датасет из предыдущей работы

In [2]:
csv = spark.read.csv('df2.csv', header=True)
csv.toPandas()

,_c0,actual_consumption,actual_pv,load_01,load_11,load_21,load_31,load_41,load_51,load_61,load_71,pv_01,pv_11,pv_21,pv_31,pv_41,pv_51,pv_61,pv_71
0,0,7.4411982744932175,0.0,6.72386622388764,6.430095676223294,8.358663519940475,19.66340391699,11.494601508241697,8.811012638329624,5.398528691373639,7.4229086317531054,0.02844860772623234,0.06840350845127914,0.07162532142860237,15.059211006002432,31.159428264392943,28.385764807496933,5.093682216781791,0.07190790163379673
1,1,7.4411982744932175,0.0,7.139108644895107,7.11192898121155,8.720978610778332,11.186682888644608,11.497683463746887,8.205496524954253,7.0465821743352794,7.344687605524327,0.02844860772623234,0.06840350845127914,0.07714854369021963,20.81260201999251,31.51322850750308,26.36367199070395,1.27541188166306,0.07190790163379673
2,2,5.580898705869913,0.0,6.919577543883818,7.723555253854789,10.844341533974859,11.65662849133767,11.497683463746887,6.42694782272626,7.5880228052403655,7.356517628795883,0.02844860772623234,0.06840350845127914,1.898652480481623,22.485364521444062,32.53297822247785,21.71940680695513,0.13058476558893878,0.07190790163379673
3,3,7.4411982744932175,0.0,7.229174824131117,7.501109296516894,10.853708080050415,11.493096331610928,11.497683463746887,6.200320626139459,7.80688684449128,7.409113024240133,0.02844860772623234,0.06840350845127914,5.188703708111551,28.225208981081142,31.830537163480773,16.79590240055818,0.07190789065002044,0.07190790163379673
4,4,5.580898705869913,0.0,7.00607084262522,7.703482071481459,11.087598724879111,11.492004887612287,9.728092783737813,5.376933891294737,7.429487848166107,7.434980821349569,0.02844860772623234,0.06840350845127914,13.63653694844751,30.123460087561,30.606277958309253,8.949279990394585,0.07190789065002044,0.07190790163379673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25616,25616,5.580898705869913,0.7358128925513145,5.538349083875105,4.387326669161299,4.258180672877748,4.2738848218982985,4.46480089466733,6.870428680950607,6.995418132307458,7.024512903295138,9.170617113949723,16.933261246383605,17.852920843397975,17.113605398297963,5.893282646164616,1.3218258136663006,1.321825813666301,1.321825813666301
25617,25617,3.7205991372466087,5.238393831490761,4.034607131487385,3.6601230623268237,4.268761254214899,4.2713729373711296,5.013063858949504,6.816309966126953,7.004473455991933,8.128976156775725,10.986198120141106,17.177818468847473,24.338361441555513,14.761859592634652,3.4104827761725818,1.3218258136663006,1.321825813666301,1.321825813666301
25618,25618,5.580898705869913,8.330013487996865,4.807785608465509,4.286978812237675,4.276351245651276,4.271888640716598,6.64300266769915,6.996699189469474,6.89972160055987,8.529005452160723,14.181548750606353,17.827298527399943,25.67497685527601,15.075939945948116,1.5871974630999206,1.3218258136663006,1.321825813666301,1.321825813666301
25619,25619,3.7205991372466087,19.320681561250655,4.27097187471122,4.397530089228814,4.2738848218982985,4.46480089466733,6.870428680950607,6.995418132307458,7.024512903295138,11.42294090449782,16.91982990165169,17.852888368071557,17.113605319776653,5.893282645974759,1.3218258136658418,1.3218258136663,1.321825813666301,1.321825813666301


## Выполнение анализа датасета с помощью:
### 1. Задача регресии (RandomForest)
Линейная регрессия — это метод анализа данных, который предсказывает ценность неизвестных данных с помощью другого связанного и известного значения данных. Он математически моделирует неизвестную или зависимую переменную и известную или независимую переменную в виде линейного уравнения.
Случайный лес (Random Forest) - это метод коллективного обучения, который объединяет прогнозы из нескольких деревьев решений для получения более точного и стабильного прогноза. Это тип алгоритма контролируемого обучения, который может использоваться как для задач классификации, так и для задач регрессии.

### Подготовка данных

Мы будем использовать признаки, показавшие наилучшую 
корреляцию при выполнении разведочного анализ в предыдущей лабораторной работе.
Для реализации линейной регрессии случайного леса нужно иметь одну зависимую переменную, которая является целевой для прогнозирования, и несколько независимых переменных, которые являются факторами или признаками, влияющими на целевую переменную.
Возьмем только показатели производства фотоэлемента.а

In [3]:
data = csv.select("actual_pv", "pv_01", "pv_41", "pv_51")
data.printSchema()
data.toPandas()

root
 |-- actual_pv: string (nullable = true)
 |-- pv_01: string (nullable = true)
 |-- pv_41: string (nullable = true)
 |-- pv_51: string (nullable = true)



,actual_pv,pv_01,pv_41,pv_51
0,0.0,0.02844860772623234,31.159428264392943,28.385764807496933
1,0.0,0.02844860772623234,31.51322850750308,26.36367199070395
2,0.0,0.02844860772623234,32.53297822247785,21.71940680695513
3,0.0,0.02844860772623234,31.830537163480773,16.79590240055818
4,0.0,0.02844860772623234,30.606277958309253,8.949279990394585
...,...,...,...,...
25616,0.7358128925513145,9.170617113949723,5.893282646164616,1.3218258136663006
25617,5.238393831490761,10.986198120141106,3.4104827761725818,1.3218258136663006
25618,8.330013487996865,14.181548750606353,1.5871974630999206,1.3218258136663006
25619,19.320681561250655,16.91982990165169,1.3218258136658418,1.3218258136663


По какой-то причине столбцы с числовыми значениями распознаются как стороки, необходимо это исправить.


In [4]:
from pyspark.sql.functions import col

columns = ["pv_51","actual_pv", "pv_01", "pv_41",]

df = data.select([col(column).cast("float").alias(column) for column in columns])

df.printSchema()

root
 |-- pv_51: float (nullable = true)
 |-- actual_pv: float (nullable = true)
 |-- pv_01: float (nullable = true)
 |-- pv_41: float (nullable = true)



### Разделение данных
80% данных будут использованы для обучения, а 20% - для тестирования.

In [5]:
splits = df.randomSplit([0.8, 0.2])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 20451  Testing Rows: 5170


### Создаем пайплайн
Чтобы преобразовать фрейм данных для моделирования и получить прогнозную модель.  
VectorAssembler - объединяет несколько колонок в одну векторную колонку для использования в качестве признаков.

In [6]:
numVect = VectorAssembler(inputCols=[ "pv_01", "pv_41", "pv_51"], outputCol="features")
rf = RandomForestRegressor(featuresCol = 'features', labelCol='actual_pv')

pipeline = Pipeline(stages=[numVect, rf])

Запускаем пайплайн

In [7]:
piplineModel = pipeline.fit(train)

In [8]:
prediction = piplineModel.transform(test)
predicted = prediction.select("features", "prediction", "actual_pv")
predicted.show(20, truncate=False)

+-------------------------------------------------------------+------------------+---------+
|features                                                     |prediction        |actual_pv|
+-------------------------------------------------------------+------------------+---------+
|[8.925382614135742,-2.305992603302002,-0.6665104031562805]   |7.370438826270356 |8.035153 |
|[37.44169616699219,0.890340268611908,-0.6407503485679626]    |27.186948364597562|39.12632 |
|[11.181233406066895,-0.09963242709636688,-0.6278612017631531]|9.807922923744238 |15.343139|
|[18.381181716918945,-1.2146704196929932,-0.6019877195358276] |14.641466621233281|13.696328|
|[29.81710433959961,-0.02314542606472969,-0.5941392183303833] |26.706886459215177|26.429955|
|[9.401190757751465,0.7965759634971619,-0.5697838068008423]   |8.96086634623824  |0.0      |
|[35.83486557006836,1.1069610118865967,-0.5616244077682495]   |27.186948364597562|37.08965 |
|[16.943159103393555,-1.2250758409500122,-0.5566403865814209] |14.6414

### Метрики для оценки регрессии

In [9]:
rfEvaluator = RegressionEvaluator(predictionCol="prediction", labelCol="actual_pv", metricName="rmse")
# RMSE
rmse = rfEvaluator.evaluate(prediction)
print(f"The RMSE for the RandomForest regression model is {rmse:0.2f}")
# MSE
mse = rfEvaluator.setMetricName("mse").evaluate(prediction)
print(f"The MSE for the RandomForest regression model is {mse:0.2f}")
# R2
r2 = rfEvaluator.setMetricName("r2").evaluate(prediction)
print(f"The R2 for the RandomForest regression model is {r2:0.2f}")
# MAE
mae = rfEvaluator.setMetricName("mae").evaluate(prediction)
print(f"The MAE for the RandomForest regression model is {mae:0.2f}")

The RMSE for the RandomForest regression model is 3.82
The MSE for the RandomForest regression model is 14.60
The R2 for the RandomForest regression model is 0.91
The MAE for the RandomForest regression model is 2.41


- RMSE означает корень из среднеквадратичной ошибки. Это квадратный корень из среднего квадрата разности между прогнозируемыми и фактическими значениями. Он измеряет, насколько близки прогнозы к фактическим значениям. Чем ниже RMSE, тем лучше модель. RMSE равный 3.63 означает, что в среднем прогнозы модели отличаются от фактических значений на 3.63 единицы.
- MSE означает среднеквадратичную ошибку. Это среднее квадрата разности между прогнозируемыми и фактическими значениями. Он измеряет дисперсию ошибок прогнозирования. Чем ниже MSE, тем лучше модель. MSE равный 13.21 означает, что в среднем квадраты ошибок прогнозирования равны 13.21 единицам.
- R2 означает коэффициент детерминации. Это доля дисперсии фактических значений, которая объясняется моделью. Он измеряет, насколько хорошо модель подходит к данным. Чем выше R2, тем лучше модель. R2 равный 0.92 означает, что 92% вариации фактических значений можно объяснить моделью.
- MAE означает среднюю абсолютную ошибку. Это среднее абсолютных разностей между прогнозируемыми и фактическими значениями. Он измеряет среднюю величину ошибок прогнозирования. Чем ниже MAE, тем лучше модель. MAE равный 2.23 означает, что в среднем прогнозы вашей модели отличаются от фактических значений на 2.23 единицы.

### Параметрическая сетка для настройки модели

In [10]:
rfparam_grid = (ParamGridBuilder()
    .addGrid(rf.maxDepth, [2, 5, 20])
    .addGrid(rf.maxBins, [5, 10, 60])
    .addGrid(rf.numTrees, [5, 20, 100])
    .build())

- rf.maxDepth - это максимальная глубина дерева, то есть максимальное количество разветвлений, которые может иметь дерево.
- rf.maxBins - это максимальное количество бинов, то есть интервалов, на которые разбиваются числовые признаки при построении дерева. Чем больше бинов, тем точнее разбиение и тем лучше качество дерева. Однако слишком большое количество бинов может замедлить процесс обучения и увеличить потребление памяти.
- rf.numTrees - это количество деревьев в лесу, то есть количество отдельных моделей, которые объединяются в одну. Чем больше деревьев, тем надежнее и стабильнее лес, так как он усредняет результаты разных деревьев и снижает риск переобучения. Однако слишком большое количество деревьев может замедлить процесс обучения и предсказания, а также потребовать больше памяти.

### Создаем кросс-валидатор

Кросс-валидатор - это инструмент, который помогает оценить качество модели регрессии на разных частях данных. Он позволяет проверить, насколько хорошо модель обобщает информацию и предсказывает новые данные, которые не были использованы при обучении. 

In [11]:
rfcv = CrossValidator(estimator=pipeline, \
                    estimatorParamMaps=rfparam_grid, \
                    evaluator=RegressionEvaluator(
                                predictionCol="prediction", \
                                labelCol="actual_pv", \
                                metricName="rmse"), \
                    numFolds=5)

### Запускаем поиск оптимальных параметров

In [12]:
rfcvModel = rfcv.fit(train)

### Создаем предсказанные значения

In [13]:
rfPredictions = rfcvModel.transform(test)

### Еще раз оцениваем модель

In [14]:
# RMSE
rmse = rfEvaluator.evaluate(rfPredictions)
print(f"The RMSE for the RandomForest regression model is {rmse:0.2f}")
# MSE
mse = rfEvaluator.setMetricName("mse").evaluate(rfPredictions)
print(f"The MSE for the RandomForest regression model is {mse:0.2f}")
# R2
r2 = rfEvaluator.setMetricName("r2").evaluate(rfPredictions)
print(f"The R2 for the RandomForest regression model is {r2:0.2f}")
# MAE
mae = rfEvaluator.setMetricName("mae").evaluate(rfPredictions)
print(f"The MAE for the RandomForest regression model is {mae:0.2f}")

The RMSE for the RandomForest regression model is 1.32
The MSE for the RandomForest regression model is 6.00
The R2 for the RandomForest regression model is 0.96
The MAE for the RandomForest regression model is 1.32


## 2. Задачи бинарной классификации
Задача классификации LogisticRegression - это задача машинного обучения с учителем, в которой целью является предсказать вероятность того, что объект принадлежит к определенному классу или нет. LogisticRegression использует логистическую функцию, также известную как сигмоидная функция, которая принимает любое вещественное значение x и выдает вероятность между 0 и 1.

In [15]:
data.describe().toPandas()

,summary,actual_pv,pv_01,pv_41,pv_51
0,count,25621,25621,25621,25621
1,mean,9.811047757299537,10.534873283634475,8.30447392585881,5.8141524496216475
2,stddev,12.589700266769716,12.450625177283769,10.891728770482338,9.46011270693771
3,min,0.0,-0.003553029195525353,-0.0002097987869169214,-0.0010594620845204472
4,max,9.999499867104516,9.991739389584927,9.99639449544998,9.986259665969685


В качестве бинароного параметра выделим показатель прогнозируемого производства фотоэлемента pv_01, так как он показал наивысшее влияние в разведочном анализе.

In [16]:
data = data.select("actual_pv", "pv_41", "pv_51",((col("pv_01") > 9.8).cast("Int").alias("label")))

In [17]:
data.show(20)

+------------------+-------------------+-------------------+-----+
|         actual_pv|              pv_41|              pv_51|label|
+------------------+-------------------+-------------------+-----+
|               0.0| 31.159428264392943| 28.385764807496933|    0|
|               0.0|  31.51322850750308|  26.36367199070395|    0|
|               0.0|  32.53297822247785|  21.71940680695513|    0|
|               0.0| 31.830537163480773|  16.79590240055818|    0|
|               0.0| 30.606277958309253|  8.949279990394585|    0|
|               0.0|  23.70581128332166|0.41051886628682555|    0|
|               0.0| 21.719405117715983|0.13058462937504467|    0|
|               0.0|  8.949278301155436|0.07190775443612633|    0|
| 1.524589958756942|0.07189833728181722|0.07190713128610234|    0|
| 3.647477720687153|0.07187518744920235|0.07190526457063594|    0|
| 9.464549794309091|0.07198841027091163| 0.0719143944332984|    1|
|20.295890280004258| 0.0719549570413601| 0.0719116968906404|  

In [18]:
columns = ["pv_51","actual_pv", "label", "pv_41",]

data = data.select([col(column).cast("float").alias(column) for column in columns])

data.printSchema()

root
 |-- pv_51: float (nullable = true)
 |-- actual_pv: float (nullable = true)
 |-- label: float (nullable = true)
 |-- pv_41: float (nullable = true)



In [19]:
splits = data.randomSplit([0.8, 0.2])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 20535  Testing Rows: 5086


### Определяем пайплайн **VectorAssemble**, объединяющий категориальные признаки в единый вектоmodel.

In [20]:
numVect = VectorAssembler(inputCols=[ "actual_pv", "pv_41", "pv_51"], outputCol="features")
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)
pipeline = Pipeline(stages=[numVect, lr])

In [21]:
piplineModel = pipeline.fit(train)

In [22]:
prediction = piplineModel.transform(test)
predicted = prediction.select("features", "prediction", "trueLabel")
predicted.show(30, truncate=False)

+--------------------------------------------------------------+----------+---------+
|features                                                      |prediction|trueLabel|
+--------------------------------------------------------------+----------+---------+
|[37.471519470214844,-1.0673669576644897,-0.879988968372345]   |1.0       |1.0      |
|[8.03515338897705,-2.305992603302002,-0.6665104031562805]     |0.0       |0.0      |
|[35.788387298583984,0.667364239692688,-0.5788235068321228]    |1.0       |1.0      |
|[27.04051971435547,-0.10895775258541107,-0.5614886283874512]  |1.0       |1.0      |
|[38.19975280761719,0.29762667417526245,-0.5228311419487]      |1.0       |1.0      |
|[33.439552307128906,0.7712415456771851,-0.5016792416572571]   |1.0       |1.0      |
|[33.439552307128906,0.7712415456771851,-0.5016792416572571]   |1.0       |1.0      |
|[4.644845962524414,-0.9471989870071411,-0.4893670976161957]   |0.0       |0.0      |
|[32.38759994506836,0.6399953365325928,-0.479968994855

### Оценка прогноза

In [23]:
tp = float(predicted.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND truelabel == 1").count())
pr = tp / (tp + fp)
re = tp / (tp + fn)
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", pr),
 ("Recall", re),
 ("F1", 2*pr*re/(re+pr))],["metric", "value"])
metrics.show()

+---------+------------------+
|   metric|             value|
+---------+------------------+
|       TP|            1691.0|
|       FP|              36.0|
|       TN|            3026.0|
|       FN|             333.0|
|Precision| 0.979154603358425|
|   Recall|0.8354743083003953|
|       F1| 0.901626233004532|
+---------+------------------+



### Оценка модели с использованием метрики AUR

In [24]:
evaluator = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
aur = evaluator.evaluate(prediction)
print ("AUR = ", aur)

#Прогноз основан на необработанной оценке прогноза, которая описывает помеченную точку логистической функции. 
# Это необработанное предсказание затем преобразуется в предсказанную метку 0 или 1 на основе вектора вероятности, 
# который указывает достоверность для каждого возможного значения метки (в данном случае 0 и 1).
# В качестве прогноза выбирается значение с наибольшей достоверностью.
prediction.select("rawPrediction", "probability", "prediction", "trueLabel").show(20, truncate=True)

AUR =  0.9895755344746127
+--------------------+--------------------+----------+---------+
|       rawPrediction|         probability|prediction|trueLabel|
+--------------------+--------------------+----------+---------+
|[-1.4753880564454...|[0.18612503697799...|       1.0|      1.0|
|[0.11386044745981...|[0.52843439939345...|       0.0|      0.0|
|[-1.3196326700300...|[0.21087941410420...|       1.0|      1.0|
|[-0.8614684018734...|[0.29703264574582...|       1.0|      1.0|
|[-1.4625471687173...|[0.18807805150952...|       1.0|      1.0|
|[-1.1847544032062...|[0.23419841897169...|       1.0|      1.0|
|[-1.1847544032062...|[0.23419841897169...|       1.0|      1.0|
|[0.34837806971256...|[0.58622420987045...|       0.0|      0.0|
|[-1.1302949040010...|[0.24410668152267...|       1.0|      1.0|
|[-1.1278596710304...|[0.24455630729087...|       1.0|      1.0|
|[-1.2049640787068...|[0.23059331186671...|       1.0|      1.0|
|[-0.7170434550425...|[0.32804436622828...|       1.0|      1.0|

### Подбор гиперпарамтеров
Чтобы найти наиболее эффективные параметры, мы можем использовать класс CrossValidator для оценки каждой комбинации параметров, определенных в ParameterGrid, по нескольким сгибам данных, разделенных на наборы данных для обучения и проверки.

In [25]:
paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.3, 0.1]).addGrid(lr.maxIter, [10, 5]).addGrid(lr.threshold, 
                                                                                            [0.4, 0.3]).build()
cv = CrossValidator(estimator=pipeline, evaluator=BinaryClassificationEvaluator(), estimatorParamMaps=paramGrid, 
                    numFolds=2)

model = cv.fit(train)

In [26]:
newPrediction = model.transform(test)
newPredicted = prediction.select("features", "prediction", "trueLabel")
newPredicted.show()

+--------------------+----------+---------+
|            features|prediction|trueLabel|
+--------------------+----------+---------+
|[37.4715194702148...|       1.0|      1.0|
|[8.03515338897705...|       0.0|      0.0|
|[35.7883872985839...|       1.0|      1.0|
|[27.0405197143554...|       1.0|      1.0|
|[38.1997528076171...|       1.0|      1.0|
|[33.4395523071289...|       1.0|      1.0|
|[33.4395523071289...|       1.0|      1.0|
|[4.64484596252441...|       0.0|      0.0|
|[32.3875999450683...|       1.0|      1.0|
|[32.1546592712402...|       1.0|      1.0|
|[33.6467247009277...|       1.0|      1.0|
|[24.1049518585205...|       1.0|      1.0|
|[24.1049518585205...|       1.0|      1.0|
|[34.8643951416015...|       1.0|      1.0|
|[4.30811119079589...|       0.0|      1.0|
|[2.76646232604980...|       0.0|      1.0|
|[11.1467390060424...|       0.0|      1.0|
|[5.46446704864501...|       0.0|      0.0|
|[9.09392642974853...|       0.0|      1.0|
|[0.0,2.4793035984...|       0.0

## Оцениваем прогноз после изменения

In [27]:
tp2 = float(newPrediction.filter("prediction == 1.0 AND truelabel == 1").count())
fp2 = float(newPrediction.filter("prediction == 1.0 AND truelabel == 0").count())
tn2 = float(newPrediction.filter("prediction == 0.0 AND truelabel == 0").count())
fn2 = float(newPrediction.filter("prediction == 0.0 AND truelabel == 1").count())
pr2 = tp2 / (tp2 + fp2)
re2 = tp2 / (tp2 + fn2)
metrics2 = spark.createDataFrame([
 ("TP", tp2),
 ("FP", fp2),
 ("TN", tn2),
 ("FN", fn2),
 ("Precision", pr2),
 ("Recall", re2),
 ("F1", 2*pr2*re2/(re2+pr2))],["metric", "value"])
metrics2.show()

+---------+------------------+
|   metric|             value|
+---------+------------------+
|       TP|            2003.0|
|       FP|             453.0|
|       TN|            2609.0|
|       FN|              21.0|
|Precision|0.8155537459283387|
|   Recall|0.9896245059288538|
|       F1|0.8941964285714284|
+---------+------------------+



In [28]:
evaluator2 = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="prediction", metricName="areaUnderROC")
aur2 = evaluator.evaluate(prediction)
print( "AUR2 = ", aur2)

AUR2 =  0.9895771480315891
